In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt


def create_cycles(df):
    # Criar uma máscara onde a válvula está aberta
    mask_open = df['valve_open'] == 0

    # Aplicar a máscara e identificar blocos consecutivos onde a válvula está aberta
    df_open = df[mask_open].copy()
    df_open['cycle'] = (df_open.index.to_series().diff() != 1).cumsum()

    # Separar os ciclos (apenas quando a válvula está aberta)
    open_cycles = [g.reset_index(drop=True) for _, g in df_open.groupby('cycle')]

    X = pd.concat(open_cycles)
    ciclos_completos = X[~X['cycle'].isin([X['cycle'].min(),X['cycle'].max()])]
    return ciclos_completos


def plot_samples(df, number=10):
    """
    Plota até `number` curvas da tabela `df`, assumindo que cada coluna é uma amostra
    e cada linha representa um tempo.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    colunas = df.columns
    for i in range(0,len(colunas),number):
        next_step = min(i + number,max(colunas))
        df[colunas[i:next_step]].mean(axis=1).plot(label=f'ciclo {i} a {next_step-1}')
    ax.legend()
    return ax

In [ ]:
df_exp_4 = pd.read_csv('./sensor_log_4.csv')
df_exp_5 = pd.read_csv('./sensor_log_5.csv')
exp_4 = create_cycles(df_exp_4)
exp_5 =  create_cycles(df_exp_5)

In [ ]:
pivot_exp_4 = pd.pivot_table(exp_4,index=exp_4.index,columns='cycle',values='sensor_value') * -1
pivot_exp_5 = pd.pivot_table(exp_5,index=exp_5.index,columns='cycle',values='sensor_value') *-1

In [ ]:
# Exemplo de uso:
plot = plot_samples(pivot_exp_4, number=20)
plot.set_xlabel('Tempo em (s)')
plot.set_ylabel('Pressão em (kPa)')
plt.show()

In [ ]:
min_60_exp_4 = pivot_exp_4[pivot_exp_4.columns[3:20]].mean(axis=1)
min_60_exp_5 = pivot_exp_5[pivot_exp_5.columns[3:20]].mean(axis=1)

min_60_exp_4_all = pivot_exp_4[pivot_exp_4.columns[:]].mean(axis=1)
min_60_exp_5_all = pivot_exp_5[pivot_exp_5.columns[:]].mean(axis=1)

In [ ]:

import matplotlib.pyplot as plt

# Gráfico 1: Ambos experimentos "all" no mesmo gráfico
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Primeiro subplot - ambas as linhas "all" juntas
min_60_exp_4_all.plot(ax=ax1, label='Experimento 4 - All')
min_60_exp_5_all.plot(ax=ax1, label='Experimento 5 - All')
ax1.set_title('Comparação - Todos os dados')
ax2.set_xlabel('Tempo em (s)')
ax2.set_ylabel('Pressão em (Kpa)')
ax1.legend()

# Segundo subplot - ambas as linhas normais juntas
min_60_exp_4.plot(ax=ax2, label='Experimento 4')
min_60_exp_5.plot(ax=ax2, label='Experimento 5')
ax2.set_title('Comparação - Primeiros 20 ciclos')
ax2.set_xlabel('Tempo em (s)')
ax2.set_ylabel('Pressão em (Kpa)')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
from sklearn.metrics import r2_score

# Simulação de dados (substitua por seus dados reais)
x = min_60_exp_4.index
y = min_60_exp_4.values

# Modelo exponencial saturado melhorado
def modelo_exponencial_saturado(x, a, b, c=0):
    """
    Modelo: y = a * (1 - exp(-b * x)) + c
    a: valor de saturação (assíntota)
    b: taxa de crescimento
    c: offset inicial (opcional)
    """
    return a * (1 - np.exp(-b * x)) + c

# Modelo alternativo: Michaelis-Menten
def modelo_michaelis_menten(x, vmax, km):
    """
    Modelo: y = (vmax * x) / (km + x)
    vmax: valor máximo (saturação)
    km: constante de meia-saturação
    """
    return (vmax * x) / (km + x)

# Modelo logístico
def modelo_logistico(x, L, k, x0):
    """
    Modelo: y = L / (1 + exp(-k * (x - x0)))
    L: valor máximo
    k: taxa de crescimento
    x0: ponto de inflexão
    """
    return L / (1 + np.exp(-k * (x - x0)))

# Estimativas iniciais mais inteligentes
def estimar_parametros_iniciais(x, y):
    """Estima parâmetros iniciais baseado nos dados"""
    a_inicial = np.max(y) * 1.1  # 10% acima do máximo observado
    
    # Estima b baseado na inclinação inicial
    if len(y) > 1:
        inclinacao_inicial = (y[1] - y[0]) / (x[1] - x[0])
        b_inicial = inclinacao_inicial / a_inicial
    else:
        b_inicial = 0.01
    
    return a_inicial, max(b_inicial, 0.001)  # garante b positivo

# Obter estimativas iniciais
a_init, b_init = estimar_parametros_iniciais(x, y)

print("=== COMPARAÇÃO DE MODELOS ===\n")

# Modelo 1: Exponencial saturado simples
try:
    params1, cov1 = curve_fit(
        lambda x, a, b: modelo_exponencial_saturado(x, a, b, 0),
        x, y, 
        p0=[a_init, b_init],
        bounds=([0, 0], [np.inf, np.inf]),
        maxfev=5000
    )
    a1, b1 = params1
    erro_a1, erro_b1 = np.sqrt(np.diag(cov1))
    y_pred1 = modelo_exponencial_saturado(x, a1, b1, 0)
    r2_1 = r2_score(y, y_pred1)
    aic1 = len(y) * np.log(np.sum((y - y_pred1)**2)/len(y)) + 2 * len(params1)
    
    print(f"Modelo 1 - Exponencial Saturado: y = {a1:.4f} * (1 - exp(-{b1:.6f} * x))")
    print(f"R² = {r2_1:.4f}, AIC = {aic1:.2f}\n")
    
except Exception as e:
    print(f"Erro no Modelo 1: {e}")
    params1, r2_1 = None, 0

# Modelo 2: Exponencial saturado com offset
try:
    params2, cov2 = curve_fit(
        modelo_exponencial_saturado,
        x, y, 
        p0=[a_init, b_init, np.min(y)],
        maxfev=5000
    )
    a2, b2, c2 = params2
    y_pred2 = modelo_exponencial_saturado(x, a2, b2, c2)
    r2_2 = r2_score(y, y_pred2)
    aic2 = len(y) * np.log(np.sum((y - y_pred2)**2)/len(y)) + 2 * len(params2)
    
    print(f"Modelo 2 - Exponencial c/ Offset: y = {a2:.4f} * (1 - exp(-{b2:.6f} * x)) + {c2:.4f}")
    print(f"R² = {r2_2:.4f}, AIC = {aic2:.2f}\n")
    
except Exception as e:
    print(f"Erro no Modelo 2: {e}")
    params2, r2_2 = None, 0

# Modelo 3: Michaelis-Menten
try:
    params3, cov3 = curve_fit(
        modelo_michaelis_menten,
        x, y, 
        p0=[np.max(y), np.median(x)],
        bounds=([0, 0], [np.inf, np.inf]),
        maxfev=5000
    )
    vmax, km = params3
    y_pred3 = modelo_michaelis_menten(x, vmax, km)
    r2_3 = r2_score(y, y_pred3)
    aic3 = len(y) * np.log(np.sum((y - y_pred3)**2)/len(y)) + 2 * len(params3)
    
    print(f"Modelo 3 - Michaelis-Menten: y = ({vmax:.4f} * x) / ({km:.2f} + x)")
    print(f"R² = {r2_3:.4f}, AIC = {aic3:.2f}\n")
    
except Exception as e:
    print(f"Erro no Modelo 3: {e}")
    params3, r2_3 = None, 0

# Selecionar melhor modelo
modelos = [
    ("Exponencial Saturado", params1, r2_1, y_pred1 if 'y_pred1' in locals() else None),
    ("Exponencial c/ Offset", params2, r2_2, y_pred2 if 'y_pred2' in locals() else None),
    ("Michaelis-Menten", params3, r2_3, y_pred3 if 'y_pred3' in locals() else None)
]

melhor_modelo = max(modelos, key=lambda x: x[2] if x[2] is not None else 0)
print(f"=== MELHOR MODELO: {melhor_modelo[0]} (R² = {melhor_modelo[2]:.4f}) ===\n")

# Plot melhorado
plt.figure(figsize=(12, 8))

# Subplot 1: Comparação dos modelos
plt.subplot(2, 2, 1)
plt.scatter(x, y, alpha=0.7, s=30, label="Dados", color='blue')

cores = ['orange', 'red', 'green']
for i, (nome, params, r2, y_pred) in enumerate(modelos):
    if y_pred is not None:
        plt.plot(x, y_pred, '--', linewidth=2, color=cores[i], 
                label=f"{nome} (R²={r2:.3f})")

plt.xlabel("x")
plt.ylabel("y")
plt.title("Comparação de Modelos")
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Resíduos do melhor modelo
plt.subplot(2, 2, 2)
if melhor_modelo[3] is not None:
    residuos = y - melhor_modelo[3]
    plt.scatter(melhor_modelo[3], residuos, alpha=0.7)
    plt.axhline(y=0, color='red', linestyle='--')
    plt.xlabel("Valores Preditos")
    plt.ylabel("Resíduos")
    plt.title(f"Resíduos - {melhor_modelo[0]}")
    plt.grid(True, alpha=0.3)

# Subplot 3: Q-Q plot dos resíduos
plt.subplot(2, 2, 3)
if melhor_modelo[3] is not None:
    from scipy import stats
    stats.probplot(residuos, dist="norm", plot=plt)
    plt.title("Q-Q Plot dos Resíduos")
    plt.grid(True, alpha=0.3)

# Subplot 4: Histograma dos resíduos
plt.subplot(2, 2, 4)
if melhor_modelo[3] is not None:
    plt.hist(residuos, bins=20, alpha=0.7, edgecolor='black')
    plt.xlabel("Resíduos")
    plt.ylabel("Frequência")
    plt.title("Distribuição dos Resíduos")
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabela de parâmetros do melhor modelo
if melhor_modelo[0] == "Exponencial Saturado" and params1 is not None:
    summary_df = pd.DataFrame({
        "Parâmetro": ["a (saturação)", "b (taxa)"],
        "Estimativa": [a1, b1],
        "Erro padrão": [erro_a1, erro_b1],
        "Inferior 95%": [a1 - 1.96 * erro_a1, b1 - 1.96 * erro_b1],
        "Superior 95%": [a1 + 1.96 * erro_a1, b1 + 1.96 * erro_b1]
    })
elif melhor_modelo[0] == "Exponencial c/ Offset" and params2 is not None:
    erro_a2, erro_b2, erro_c2 = np.sqrt(np.diag(cov2))
    summary_df = pd.DataFrame({
        "Parâmetro": ["a (saturação)", "b (taxa)", "c (offset)"],
        "Estimativa": [a2, b2, c2],
        "Erro padrão": [erro_a2, erro_b2, erro_c2],
        "Inferior 95%": [a2 - 1.96 * erro_a2, b2 - 1.96 * erro_b2, c2 - 1.96 * erro_c2],
        "Superior 95%": [a2 + 1.96 * erro_a2, b2 + 1.96 * erro_b2, c2 + 1.96 * erro_c2]
    })
elif melhor_modelo[0] == "Michaelis-Menten" and params3 is not None:
    erro_vmax, erro_km = np.sqrt(np.diag(cov3))
    summary_df = pd.DataFrame({
        "Parâmetro": ["Vmax (saturação)", "Km (meia-saturação)"],
        "Estimativa": [vmax, km],
        "Erro padrão": [erro_vmax, erro_km],
        "Inferior 95%": [vmax - 1.96 * erro_vmax, km - 1.96 * erro_km],
        "Superior 95%": [vmax + 1.96 * erro_vmax, km + 1.96 * erro_km]
    })

print("\n=== PARÂMETROS DO MELHOR MODELO ===")
print(summary_df.to_string(index=False, float_format='%.6f'))

# Métricas de qualidade do ajuste
print(f"\n=== MÉTRICAS DE QUALIDADE ===")
print(f"R² = {melhor_modelo[2]:.4f}")
if melhor_modelo[3] is not None:
    rmse = np.sqrt(np.mean((y - melhor_modelo[3])**2))
    mae = np.mean(np.abs(y - melhor_modelo[3]))
    print(f"RMSE = {rmse:.6f}")
    print(f"MAE = {mae:.6f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
from sklearn.metrics import r2_score

# Simulação de dados (substitua por seus dados reais)
x = min_60_exp_5.index
y = min_60_exp_5.values

# Modelo exponencial saturado melhorado
def modelo_exponencial_saturado(x, a, b, c=0):
    """
    Modelo: y = a * (1 - exp(-b * x)) + c
    a: valor de saturação (assíntota)
    b: taxa de crescimento
    c: offset inicial (opcional)
    """
    return a * (1 - np.exp(-b * x)) + c

# Modelo alternativo: Michaelis-Menten
def modelo_michaelis_menten(x, vmax, km):
    """
    Modelo: y = (vmax * x) / (km + x)
    vmax: valor máximo (saturação)
    km: constante de meia-saturação
    """
    return (vmax * x) / (km + x)

# Modelo logístico
def modelo_logistico(x, L, k, x0):
    """
    Modelo: y = L / (1 + exp(-k * (x - x0)))
    L: valor máximo
    k: taxa de crescimento
    x0: ponto de inflexão
    """
    return L / (1 + np.exp(-k * (x - x0)))

# Estimativas iniciais mais inteligentes
def estimar_parametros_iniciais(x, y):
    """Estima parâmetros iniciais baseado nos dados"""
    a_inicial = np.max(y) * 1.1  # 10% acima do máximo observado
    
    # Estima b baseado na inclinação inicial
    if len(y) > 1:
        inclinacao_inicial = (y[1] - y[0]) / (x[1] - x[0])
        b_inicial = inclinacao_inicial / a_inicial
    else:
        b_inicial = 0.01
    
    return a_inicial, max(b_inicial, 0.001)  # garante b positivo

# Obter estimativas iniciais
a_init, b_init = estimar_parametros_iniciais(x, y)

print("=== COMPARAÇÃO DE MODELOS ===\n")

# Modelo 1: Exponencial saturado simples
try:
    params1, cov1 = curve_fit(
        lambda x, a, b: modelo_exponencial_saturado(x, a, b, 0),
        x, y, 
        p0=[a_init, b_init],
        bounds=([0, 0], [np.inf, np.inf]),
        maxfev=5000
    )
    a1, b1 = params1
    erro_a1, erro_b1 = np.sqrt(np.diag(cov1))
    y_pred1 = modelo_exponencial_saturado(x, a1, b1, 0)
    r2_1 = r2_score(y, y_pred1)
    aic1 = len(y) * np.log(np.sum((y - y_pred1)**2)/len(y)) + 2 * len(params1)
    
    print(f"Modelo 1 - Exponencial Saturado: y = {a1:.4f} * (1 - exp(-{b1:.6f} * x))")
    print(f"R² = {r2_1:.4f}, AIC = {aic1:.2f}\n")
    
except Exception as e:
    print(f"Erro no Modelo 1: {e}")
    params1, r2_1 = None, 0

# Modelo 2: Exponencial saturado com offset
try:
    params2, cov2 = curve_fit(
        modelo_exponencial_saturado,
        x, y, 
        p0=[a_init, b_init, np.min(y)],
        maxfev=5000
    )
    a2, b2, c2 = params2
    y_pred2 = modelo_exponencial_saturado(x, a2, b2, c2)
    r2_2 = r2_score(y, y_pred2)
    aic2 = len(y) * np.log(np.sum((y - y_pred2)**2)/len(y)) + 2 * len(params2)
    
    print(f"Modelo 2 - Exponencial c/ Offset: y = {a2:.4f} * (1 - exp(-{b2:.6f} * x)) + {c2:.4f}")
    print(f"R² = {r2_2:.4f}, AIC = {aic2:.2f}\n")
    
except Exception as e:
    print(f"Erro no Modelo 2: {e}")
    params2, r2_2 = None, 0

# Modelo 3: Michaelis-Menten
try:
    params3, cov3 = curve_fit(
        modelo_michaelis_menten,
        x, y, 
        p0=[np.max(y), np.median(x)],
        bounds=([0, 0], [np.inf, np.inf]),
        maxfev=5000
    )
    vmax, km = params3
    y_pred3 = modelo_michaelis_menten(x, vmax, km)
    r2_3 = r2_score(y, y_pred3)
    aic3 = len(y) * np.log(np.sum((y - y_pred3)**2)/len(y)) + 2 * len(params3)
    
    print(f"Modelo 3 - Michaelis-Menten: y = ({vmax:.4f} * x) / ({km:.2f} + x)")
    print(f"R² = {r2_3:.4f}, AIC = {aic3:.2f}\n")
    
except Exception as e:
    print(f"Erro no Modelo 3: {e}")
    params3, r2_3 = None, 0

# Selecionar melhor modelo
modelos = [
    ("Exponencial Saturado", params1, r2_1, y_pred1 if 'y_pred1' in locals() else None),
    ("Exponencial c/ Offset", params2, r2_2, y_pred2 if 'y_pred2' in locals() else None),
    ("Michaelis-Menten", params3, r2_3, y_pred3 if 'y_pred3' in locals() else None)
]

melhor_modelo = max(modelos, key=lambda x: x[2] if x[2] is not None else 0)
print(f"=== MELHOR MODELO: {melhor_modelo[0]} (R² = {melhor_modelo[2]:.4f}) ===\n")

# Plot melhorado
plt.figure(figsize=(12, 8))

# Subplot 1: Comparação dos modelos
plt.subplot(2, 2, 1)
plt.scatter(x, y, alpha=0.7, s=30, label="Dados", color='blue')

cores = ['orange', 'red', 'green']
for i, (nome, params, r2, y_pred) in enumerate(modelos):
    if y_pred is not None:
        plt.plot(x, y_pred, '--', linewidth=2, color=cores[i], 
                label=f"{nome} (R²={r2:.3f})")

plt.xlabel("x")
plt.ylabel("y")
plt.title("Comparação de Modelos")
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Resíduos do melhor modelo
plt.subplot(2, 2, 2)
if melhor_modelo[3] is not None:
    residuos = y - melhor_modelo[3]
    plt.scatter(melhor_modelo[3], residuos, alpha=0.7)
    plt.axhline(y=0, color='red', linestyle='--')
    plt.xlabel("Valores Preditos")
    plt.ylabel("Resíduos")
    plt.title(f"Resíduos - {melhor_modelo[0]}")
    plt.grid(True, alpha=0.3)

# Subplot 3: Q-Q plot dos resíduos
plt.subplot(2, 2, 3)
if melhor_modelo[3] is not None:
    from scipy import stats
    stats.probplot(residuos, dist="norm", plot=plt)
    plt.title("Q-Q Plot dos Resíduos")
    plt.grid(True, alpha=0.3)

# Subplot 4: Histograma dos resíduos
plt.subplot(2, 2, 4)
if melhor_modelo[3] is not None:
    plt.hist(residuos, bins=20, alpha=0.7, edgecolor='black')
    plt.xlabel("Resíduos")
    plt.ylabel("Frequência")
    plt.title("Distribuição dos Resíduos")
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabela de parâmetros do melhor modelo
if melhor_modelo[0] == "Exponencial Saturado" and params1 is not None:
    summary_df = pd.DataFrame({
        "Parâmetro": ["a (saturação)", "b (taxa)"],
        "Estimativa": [a1, b1],
        "Erro padrão": [erro_a1, erro_b1],
        "Inferior 95%": [a1 - 1.96 * erro_a1, b1 - 1.96 * erro_b1],
        "Superior 95%": [a1 + 1.96 * erro_a1, b1 + 1.96 * erro_b1]
    })
elif melhor_modelo[0] == "Exponencial c/ Offset" and params2 is not None:
    erro_a2, erro_b2, erro_c2 = np.sqrt(np.diag(cov2))
    summary_df = pd.DataFrame({
        "Parâmetro": ["a (saturação)", "b (taxa)", "c (offset)"],
        "Estimativa": [a2, b2, c2],
        "Erro padrão": [erro_a2, erro_b2, erro_c2],
        "Inferior 95%": [a2 - 1.96 * erro_a2, b2 - 1.96 * erro_b2, c2 - 1.96 * erro_c2],
        "Superior 95%": [a2 + 1.96 * erro_a2, b2 + 1.96 * erro_b2, c2 + 1.96 * erro_c2]
    })
elif melhor_modelo[0] == "Michaelis-Menten" and params3 is not None:
    erro_vmax, erro_km = np.sqrt(np.diag(cov3))
    summary_df = pd.DataFrame({
        "Parâmetro": ["Vmax (saturação)", "Km (meia-saturação)"],
        "Estimativa": [vmax, km],
        "Erro padrão": [erro_vmax, erro_km],
        "Inferior 95%": [vmax - 1.96 * erro_vmax, km - 1.96 * erro_km],
        "Superior 95%": [vmax + 1.96 * erro_vmax, km + 1.96 * erro_km]
    })

print("\n=== PARÂMETROS DO MELHOR MODELO ===")
print(summary_df.to_string(index=False, float_format='%.6f'))

# Métricas de qualidade do ajuste
print(f"\n=== MÉTRICAS DE QUALIDADE ===")
print(f"R² = {melhor_modelo[2]:.4f}")
if melhor_modelo[3] is not None:
    rmse = np.sqrt(np.mean((y - melhor_modelo[3])**2))
    mae = np.mean(np.abs(y - melhor_modelo[3]))
    print(f"RMSE = {rmse:.6f}")
    print(f"MAE = {mae:.6f}")